In [1]:
import sqlite3
import pandas as pd
import numpy as np
import sys
import glob, os
from pyteomics import mgf

In [2]:
RT_LIMIT_LOWER = 0
RT_LIMIT_UPPER = 3000

In [3]:
RT_BUFFER_WIDTH = 0

In [5]:
BASE_MAXQUANT_DIR = '/Users/darylwilding-mcbride/Downloads/analyses/combined'
MAXQUANT_TXT_DIR = '{}/txt'.format(BASE_MAXQUANT_DIR)
ALLPEPTIDES_FILENAME = '{}/allPeptides.txt'.format(MAXQUANT_TXT_DIR)
APL_DIR = '{}/andromeda'.format(BASE_MAXQUANT_DIR)

In [6]:
BASE_NAME = 'P3795'
MGF_FILE_NAME = '{}/{}-mq.mgf'.format(BASE_MAXQUANT_DIR, BASE_NAME)

In [7]:
MIN_ISOTOPE_CORRELATION = 0.9

allpeptides_df = pd.read_csv(ALLPEPTIDES_FILENAME, sep='\t')
allpeptides_df.rename(columns={'Number of isotopic peaks':'isotope_count', 'm/z':'mz', 'Number of data points':'number_data_points', 'Intensity':'intensity', 'Ion mobility index':'scan', 'Ion mobility index length':'scan_length', 'Ion mobility index length (FWHM)':'scan_length_fwhm', 'Retention time':'rt', 'Retention length':'rt_length', 'Retention length (FWHM)':'rt_length_fwhm', 'Charge':'charge_state', 'Number of pasef MS/MS':'number_pasef_ms2_ids', 'Pasef MS/MS IDs':'pasef_msms_ids', 'MS/MS scan number':'msms_scan_number', 'Isotope correlation':'isotope_correlation'}, inplace=True)
# allpeptides_df = allpeptides_df[allpeptides_df.intensity.notnull() & (allpeptides_df.number_pasef_ms2_ids > 0) & (allpeptides_df.msms_scan_number >= 0) & allpeptides_df.pasef_msms_ids.notnull() & (allpeptides_df.isotope_correlation >= MIN_ISOTOPE_CORRELATION) & (allpeptides_df.rt >= RT_LIMIT_LOWER+RT_BUFFER_WIDTH) & (allpeptides_df.rt <= RT_LIMIT_UPPER-RT_BUFFER_WIDTH)].copy()
allpeptides_df = allpeptides_df[allpeptides_df.intensity.notnull() & (allpeptides_df.number_pasef_ms2_ids > 0) & (allpeptides_df.msms_scan_number >= 0) & allpeptides_df.pasef_msms_ids.notnull() & (allpeptides_df.isotope_correlation >= MIN_ISOTOPE_CORRELATION)].copy()


In [8]:
allpeptides_df.msms_scan_number = allpeptides_df.msms_scan_number.apply(lambda x: int(x))

In [9]:
allpeptides_df.msms_scan_number.max()

28687

In [10]:
allpeptides_df.tail()

,Raw file,charge_state,mz,Mass,Resolution,number_data_points,Number of frames,isotope_count,isotope_correlation,Mass fractional part,...,Min frame index,Max frame index,scan,scan_length,scan_length_fwhm,intensity,Intensities,number_pasef_ms2_ids,pasef_msms_ids,msms_scan_number
128924,P3795_withCompound_Slot1-15_1_3452,2,379.17882,756.34308,25425.739337,472,10,2,0.995390,0.343083,...,3763,3772,915,18,15,15209.0,NaN,4,75745;75756;75767;75777,659
128925,P3795_withCompound_Slot1-15_1_3452,3,357.16334,1068.46820,25817.435919,299,9,2,0.970008,0.468195,...,4007,4015,921,12,9,5005.6,NaN,7,100812;100825;100838;100850;100862;100874;100885,3232
128926,P3795_withCompound_Slot1-15_1_3452,3,429.83560,1286.48500,24442.451175,244,8,2,0.956107,0.484982,...,2454,2461,918,9,9,4836.1,NaN,12,10824;10833;10849;10866;10873;11121;11129;1119...,5669
128927,P3795_withCompound_Slot1-15_1_3452,1,421.25389,420.24661,24077.323974,3451,37,2,0.988118,0.246613,...,3245,3281,897,36,3,68638.0,NaN,8,44273;44277;44281;45054;45060;45066;45072;45078,32
128930,P3795_withCompound_Slot1-15_1_3452,3,405.15771,1212.45130,24077.563046,232,7,2,0.920694,0.451294,...,2455,2461,930,12,12,4524.8,NaN,3,10927;10935;10942,4822


In [19]:
allpeptides_df['Raw file'].unique()

array(['P3795_noCompound_Slot1-14_1_3450',
       'P3795_withCompound_Slot1-15_1_3452'], dtype=object)

In [11]:
allpeptides_df.columns

Index(['Raw file', 'charge_state', 'mz', 'Mass', 'Resolution',
       'number_data_points', 'Number of frames', 'isotope_count',
       'isotope_correlation', 'Mass fractional part', 'Mass deficit', 'rt',
       'rt_length', 'rt_length_fwhm', 'Min frame index', 'Max frame index',
       'scan', 'scan_length', 'scan_length_fwhm', 'intensity', 'Intensities',
       'number_pasef_ms2_ids', 'pasef_msms_ids', 'msms_scan_number'],
      dtype='object')

In [12]:
def collate_spectra_for_feature(ms1_d, ms2_df):
    # append the monoisotopic and the ms2 fragments to the list for MGF creation
    pairs_df = ms2_df[['mz', 'intensity']].copy().sort_values(by=['intensity'], ascending=False)
    spectrum = {}
    spectrum["m/z array"] = pairs_df.mz.values
    spectrum["intensity array"] = pairs_df.intensity.values
    params = {}
    params["TITLE"] = "RawFile: {} Index: {} Charge: {} FeatureIntensity: {} RtApex: {}".format(ms1_d['raw_file'], ms1_d['mq_index'], ms1_d['charge'], ms1_d['intensity'], round(ms1_d['rt_apex'],2))
    params["INSTRUMENT"] = "ESI-QUAD-TOF"
    params["PEPMASS"] = "{} {}".format(round(ms1_d['monoisotopic_mz'],6), ms1_d['intensity'])
    params["CHARGE"] = "{}+".format(ms1_d['charge'])
    params["RTINSECONDS"] = "{}".format(round(ms1_d['rt_apex'],2))
    params["SCANS"] = "{}".format(int(ms1_d['rt_apex']))
    spectrum["params"] = params
    return spectrum

In [50]:
# build a list of indexes from the APL files
ms2_peaks = []
apl_indexes = []
for file in glob.glob("{}/*.apl".format(APL_DIR)):
    with open(file, 'r') as f:
        for line in f:
            line = line.rstrip()
            if len(line) > 0:
                if line.startswith("header="):
                    line_a = line.split(' ')
                    mq_index = int(line_a[3])
                    raw_file = line_a[1]
                if line[0].isdigit():
                    line_a = line.split('\t')
                    mz = float(line_a[0])
                    intensity = round(float(line_a[1]))
                    ms2_peaks.append((mz, intensity))
                if line.startswith("peaklist end"):
                    apl_indexes.append((mq_index, ms2_peaks.copy(), raw_file))
                    del ms2_peaks[:]
                    mq_index = 0
apl_indexes_df = pd.DataFrame(apl_indexes, columns=['mq_index','ms2_peaks','raw_file'])

In [53]:
mgf_spectra = []
for idx,row in allpeptides_df.iterrows():
    mq_index = row.msms_scan_number
    ms1_d = {'monoisotopic_mass':row.Mass, 
             'charge':row.charge_state, 
             'monoisotopic_mz':row.mz, 
             'intensity':int(row.intensity), 
             'scan_apex':row.scan, 
             'rt_apex':row.rt,
             'raw_file':row['Raw file'],
             'mq_index':mq_index}
    df = apl_indexes_df[(apl_indexes_df.mq_index == mq_index) & (apl_indexes_df['raw_file'] == row['Raw file'])]
    ms2_peaks_df = pd.DataFrame(df.iloc[0].ms2_peaks, columns=['mz','intensity'])
    feature_spectra = collate_spectra_for_feature(ms1_d, ms2_peaks_df)
    mgf_spectra.append(feature_spectra)


In [54]:
len(mgf_spectra)

37493

In [55]:
allpeptides_df.msms_scan_number.nunique()

24070

In [56]:
# generate the MGF for all the features
print("generating the MGF: {}".format(MGF_FILE_NAME))
if os.path.isfile(MGF_FILE_NAME):
    os.remove(MGF_FILE_NAME)
f = mgf.write(output=MGF_FILE_NAME, spectra=mgf_spectra)


generating the MGF: /Users/darylwilding-mcbride/Downloads/analyses/combined/P3795-mq.mgf
